# LDC Implementation in Python/JAX
Written By: Sungje Park

In [1]:
import jax
from jax import numpy as jnp

import sys
sys.path.append('..')
from src.environment import *
from src.elements import *
from src.dynamics import *
from utils.test_utils import *
from src.methods import *
from tqdm import tqdm
import matplotlib.pyplot as plt

Defining Vars.

In [ ]:
rho_ref = 9 # reference density
Mu = .1 # kinematic Viscosity
Re = 100 # reynolds number
N_x = 100 # number of cells in the x dir
N_y = N_x # number of cells in the y dir
dx = 1 # spacing in x
dy = 1 # spacing in y
U_lid = Re*Mu/N_x # velocity of lid
Tau = 3*Mu+.5
dt = 1

In [3]:
cell_mesh = jnp.reshape(jnp.asarray(range(N_x*N_y)),(N_y,N_x))
node_mesh = jnp.reshape(jnp.asarray(range((N_x+1)*(N_y+1))),(N_y+1,N_x+1))
face_mesh = jnp.asarray(range(N_x*(N_y+1)+(N_x+1)*N_y))
vert_mesh = jnp.reshape(face_mesh[0:N_x*(N_y+1)],(N_y,N_x+1))
horz_mesh = jnp.reshape(face_mesh[N_x*(N_y+1):],(N_y+1,N_x))

In [4]:
dynamics = D2Q9(tau=Tau,delta_t = dt)
Element.dynamics = dynamics
Environment.dynamics = dynamics

In [5]:
def create_cell(cell_ind):
    loc = jnp.argwhere(cell_mesh == cell_ind)[0]
    x = loc[1]
    y = loc[0]
    pdf = dynamics.ones_pdf()
    face_ind = jnp.asarray([vert_mesh[y,x],
                            horz_mesh[y,x],
                            vert_mesh[y,x+1],
                            horz_mesh[y+1,x]])
    face_n = jnp.asarray([1,1,-1,-1])
    return Cell.pdf_init(pdf,face_ind,face_n)

cells = [create_cell(ind) for ind in tqdm(range(N_x*N_y))]

100%|██████████| 100/100 [00:01<00:00, 57.91it/s]


In [6]:
def create_node(node_ind):
    loc = jnp.argwhere(node_mesh==node_ind)[0]
    x = loc[1]
    y = loc[0]
    pdf = dynamics.ones_pdf()
    cell_index = []
    cell_dists = []
    cell_dist = jnp.sqrt(2)
    def cell_append(tl,tr,bl,br):
        if tl:
            cell_index.append(cell_mesh[y-1,x-1]) # top left
            cell_dists.append(cell_dist)
        if tr:
            cell_index.append(cell_mesh[y-1,x]) # top right
            cell_dists.append(cell_dist)

        if br:
            cell_index.append(cell_mesh[y,x]) # bottom right
            cell_dists.append(cell_dist)

        if bl:
            cell_index.append(cell_mesh[y,x-1])# bottom left
            cell_dists.append(cell_dist)

    if y == 0:
        type = 1
        vel = jnp.asarray([U_lid,0])
        if x!=0 and x!=N_x:
            cell_append(False,False,True,True)
        elif x==0:
            vel = jnp.asarray([0,0])
            cell_append(False,False,False,True)
        elif x==N_x:
            vel = jnp.asarray([0,0])
            cell_append(False,False,True,False)

    elif y == N_y:
        type = 1
        vel = jnp.asarray([0,0])
        if x!=0 and x!=N_x:
            cell_append(True,True,False,False)
        elif x==0:
            cell_append(False,True,False,False)
        elif x==N_x:
            cell_append(True,False,False,False)

    elif x == 0:
        type = 1
        vel = jnp.asarray([0,0])
        # since all corner cases handled
        cell_append(False,True,False,True)

    elif x == N_x:
        type = 1
        vel = jnp.asarray([0,0])
        cell_append(True,False,True,False)
    else:
        type = 0
        cell_append(True,True,True,True)

    node = Node.pdf_init(pdf,jnp.asarray(type),jnp.asarray(cell_index),jnp.asarray(cell_dists))

    if type == 1:
        node.vel = vel
    return node
nodes = [create_node(ind) for ind in tqdm(range((N_x+1)*(N_y+1)))]

100%|██████████| 121/121 [00:01<00:00, 60.72it/s]


In [7]:
node_vel = jnp.asarray([nodes[ind].vel for ind in tqdm(range((N_x+1)*(N_y+1)))])
node_vel = node_vel.reshape(N_y+1,N_x+1,2)


100%|██████████| 121/121 [00:00<00:00, 1033626.85it/s]


In [8]:
def create_face(face_ind):
    pdf = dynamics.ones_pdf()
    nodes_ind = []
    stencil_ind = []
    stencil_dists = jnp.asarray([.5,.5])
    
    if face_ind < N_x*(N_y+1): # vertical faces
        loc = jnp.argwhere(vert_mesh==face_ind)[0]
        x = loc[1]
        y = loc[0]
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x]].type,node_mesh[y,x]]))
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y+1,x]].type,node_mesh[y+1,x]]))

        if x == 0:
            stencil_ind.append(-1)
            stencil_ind.append(cell_mesh[y,x])
        elif x == N_x:
            stencil_ind.append(cell_mesh[y,x-1])
            stencil_ind.append(-1)
        else:
            stencil_ind.append(cell_mesh[y,x-1])
            stencil_ind.append(cell_mesh[y,x])

        n = jnp.asarray([1,0])
    else: # horizontal faces
        loc = jnp.argwhere(horz_mesh==face_ind)[0]
        x = loc[1]
        y = loc[0]
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x]].type,node_mesh[y,x]]))
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x+1]].type,node_mesh[y,x+1]]))

        if y == 0:
            stencil_ind.append(-1)
            stencil_ind.append(cell_mesh[y,x])
        elif y == N_y:
            stencil_ind.append(cell_mesh[y-1,x])
            stencil_ind.append(-1)
        else:
            stencil_ind.append(cell_mesh[y-1,x])
            stencil_ind.append(cell_mesh[y,x])

        n = jnp.asarray([0,1])

    face = Face.pdf_init(pdf,jnp.asarray(nodes_ind),jnp.asarray(stencil_ind),jnp.asarray(stencil_dists),n,jnp.asarray(1))
    return face

faces = [create_face(ind) for ind in tqdm(range((N_x)*(N_x+1)*2))]

100%|██████████| 220/220 [00:03<00:00, 57.20it/s]


In [9]:
env = Environment.create(cells,faces,nodes)
env.setMethods(Methods)
params,config = env.init()

In [10]:
params

{'cells': {'pdf': Array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.

In [11]:
params = env.step(params,config)
params

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.        ,  0.        , -0.01499999,  0.        ,  0.        ,
         0.08249998, -0.0675    ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02999997,  0.        ,  0.        ,
         0.16499996, -0.13499999,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02999997,  0.        ,  0.        ,
         0.16499996, -0.13499999,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02999997,  0.        ,  0.        ,
         0.16499996, -0.13499999,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02999997,  0.        ,  0.        ,
         0.16499996, -0.13499999,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02999997,  0.        ,  0.        ,
         0.16499996, -0.13499999,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02999997,  0.        ,  0.        ,
         0.16499996, -0.13499999,  0.        ,

{'cells': {'pdf': Array([[ 4.75      ,  1.        ,  1.015     ,  1.        ,  1.        ,
          -0.01999998,  0.13      ,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.03      ,  1.        ,  1.        ,
          -0.10249996,  0.19749999,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.03      ,  1.        ,  1.        ,
          -0.10249996,  0.19749999,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.03      ,  1.        ,  1.        ,
          -0.10249996,  0.19749999,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.03      ,  1.        ,  1.        ,
          -0.10249996,  0.19749999,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.03      ,  1.        ,  1.        ,
          -0.10249996,  0.19749999,  0.0625    ,  0.0625    ],
         [ 4.75      ,  1.        ,  1.03      ,  1.        ,  1.        ,
          -0.10249996,  0.19749999,  0.0625    ,  0.0625    ],
         [ 4.75   

In [12]:
params = env.step(params,config)
params

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.        ,  0.07375002, -0.00604177,  0.        ,  0.        ,
         0.1314583 ,  0.00177082,  0.        ,  0.0184375 ],
       [ 0.        ,  0.        , -0.03479171,  0.        ,  0.        ,
         0.33114573, -0.21385416,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02666664,  0.        ,  0.        ,
         0.21333325, -0.18666667,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02666664,  0.        ,  0.        ,
         0.21333325, -0.18666667,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02666664,  0.        ,  0.        ,
         0.21333325, -0.18666667,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02666664,  0.        ,  0.        ,
         0.21333325, -0.18666667,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.02666664,  0.        ,  0.        ,
         0.21333325, -0.18666667,  0.        ,

{'cells': {'pdf': Array([[ 3.810416  ,  0.8647915 ,  1.0017707 ,  1.0635415 ,  0.999479  ,
           0.17067708,  0.29411456,  0.31276038,  0.26307288],
         [ 3.8041668 ,  0.8791667 ,  1.0252084 ,  1.1291667 ,  0.9979167 ,
          -0.02322903,  0.50927085,  0.32916668,  0.26666668],
         [ 3.8041668 ,  0.8791667 ,  1.0170833 ,  1.1291667 ,  0.9979167 ,
           0.09458345,  0.48208335,  0.32916668,  0.26666668],
         [ 3.8041668 ,  0.8791667 ,  1.0170833 ,  1.1291667 ,  0.9979167 ,
           0.09458345,  0.48208335,  0.32916668,  0.26666668],
         [ 3.8041668 ,  0.8791667 ,  1.0170833 ,  1.1291667 ,  0.9979167 ,
           0.09458345,  0.48208335,  0.32916668,  0.26666668],
         [ 3.8041668 ,  0.8791667 ,  1.0170833 ,  1.1291667 ,  0.9979167 ,
           0.09458345,  0.48208335,  0.32916668,  0.26666668],
         [ 3.8041668 ,  0.8791667 ,  1.0170833 ,  1.1291667 ,  0.9979167 ,
           0.09458345,  0.48208335,  0.32916668,  0.26666668],
         [ 3.80416

In [13]:
params = env.step(params,config)
params

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.0000000e+00,  2.0755768e-01,  7.4904442e-02,  6.5625191e-02,
         5.2100420e-04,  2.4443524e-01,  1.8890432e-01,  5.2091479e-04,
         6.9221944e-02],
       [ 0.0000000e+00, -1.4375210e-02,  1.8879652e-02,  0.0000000e+00,
         2.0833015e-03,  6.2724090e-01, -3.9277670e-01, -3.2291681e-02,
         2.6614517e-02],
       [ 0.0000000e+00,  0.0000000e+00,  8.0378056e-03,  0.0000000e+00,
         2.0833015e-03,  1.0289812e-01, -2.4956411e-01, -3.2291681e-02,
         3.0208319e-02],
       [ 0.0000000e+00,  0.0000000e+00, -1.1851788e-02,  0.0000000e+00,
         2.0833015e-03,  2.7259254e-01, -2.6074076e-01, -3.2291681e-02,
         3.0208319e-02],
       [ 0.0000000e+00,  0.0000000e+00, -1.1851788e-02,  0.0000000e+00,
         2.0833015e-03,  2.7259254e-01, -2.6074076e-01, -3.2291681e-02,
         3.0208319e-02],
       [ 0.0000000e+00,  0.0000000e+00, -1.1851788e-02,  0.0000000e+00,
      

{'cells': {'pdf': Array([[ 3.911273  ,  0.64710927,  0.84615153,  1.0492502 ,  1.0117326 ,
          -0.02906667,  0.07020023,  0.27945703,  0.1428253 ],
         [ 3.9357638 ,  0.7142453 ,  0.91262174,  1.3415368 ,  1.0049909 ,
          -0.3975497 ,  0.6604986 ,  0.36711308,  0.14290431],
         [ 4.0035877 ,  0.7612268 ,  0.97634876,  1.2820603 ,  0.9870949 ,
           0.11871065,  0.52013123,  0.34108797,  0.14837962],
         [ 4.0035877 ,  0.7612268 ,  0.99623835,  1.2820603 ,  0.9870949 ,
          -0.05098377,  0.53130794,  0.34108797,  0.14837962],
         [ 4.0035877 ,  0.7612268 ,  0.99623835,  1.2820603 ,  0.9870949 ,
          -0.05098377,  0.53130794,  0.34108797,  0.14837962],
         [ 4.0035877 ,  0.7612268 ,  0.99623835,  1.2820603 ,  0.9870949 ,
          -0.05098377,  0.53130794,  0.34108797,  0.14837962],
         [ 4.0035877 ,  0.7612268 ,  0.99623835,  1.2820603 ,  0.9870949 ,
          -0.05098377,  0.53130794,  0.34108797,  0.14837962],
         [ 4.00358

In [14]:
params = env.step(params,config)
params

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00,  4.06621337e-01,  3.76103938e-01,
         2.92286515e-01, -2.87851095e-02,  5.11434615e-01,
         7.42968678e-01,  2.67709792e-02,  1.70086175e-01],
       [ 0.00000000e+00, -6.71360493e-02,  2.25653172e-01,
        -5.94764948e-02, -7.07423687e-03,  1.17850900e+00,
        -8.45220864e-01, -1.85065269e-01,  1.33735448e-01],
       [ 0.00000000e+00, -4.69814539e-02,  2.87848711e-02,
         0.00000000e+00,  1.08218193e-02, -5.07803142e-01,
        -3.04013133e-01, -1.33015037e-01,  1.16717994e-01],
       [ 0.00000000e+00,  0.00000000e+00, -5.53548336e-03,
         0.00000000e+00,  1.08218193e-02,  5.95873415e-01,
        -3.86512160e-01, -1.33015037e-01,  1.22193307e-01],
       [ 0.00000000e+00,  0.00000000e+00,  3.21919918e-02,
         0.00000000e+00,  1.08218193e-02,  3.49876285e-01,
        -3.82068276e-01, -1.33015037e-01,  1.22193307e-01],
       [ 0.00000000e+00,  0.00000

{'cells': {'pdf': Array([[ 3.3529422 ,  0.280531  ,  0.2908061 ,  0.8260304 ,  1.1099082 ,
          -0.32924753, -0.4801705 ,  0.32511213,  0.05553289],
         [ 3.478414  ,  0.47100985,  0.528642  ,  1.8978179 ,  1.0211418 ,
          -0.9434673 ,  1.166913  ,  0.69747996, -0.0097511 ],
         [ 3.9610066 ,  0.71760267,  1.0266799 ,  1.4695483 ,  0.9264329 ,
           0.70284104,  0.6415506 ,  0.47586155,  0.05259266],
         [ 3.8436222 ,  0.59750247,  0.9676057 ,  1.5471556 ,  0.95353425,
          -0.38617516,  0.72060287,  0.5146607 ,  0.03766409],
         [ 3.8436222 ,  0.59750247,  0.92987823,  1.5471556 ,  0.95353425,
          -0.14017804,  0.716159  ,  0.5146607 ,  0.03766409],
         [ 3.8436222 ,  0.59750247,  0.92987823,  1.5471556 ,  0.95353425,
          -0.14017804,  0.716159  ,  0.5146607 ,  0.03766409],
         [ 3.8436222 ,  0.59750247,  0.9118476 ,  1.5471556 ,  0.95353425,
          -0.14536561,  0.89944434,  0.5146607 ,  0.03766409],
         [ 3.81223

In [15]:
params = env.step(params,config)
params

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00,  8.06842744e-01,  1.08254123e+00,
         1.07178748e+00, -2.66813159e-01,  9.00404692e-01,
         2.40547943e+00,  2.12156668e-01,  3.00054610e-01],
       [ 0.00000000e+00, -1.90478861e-01,  8.08410108e-01,
        -4.28269506e-01, -8.45924020e-02,  2.07127476e+00,
        -2.12698460e+00, -8.00294518e-01,  5.16753554e-01],
       [ 0.00000000e+00, -2.46592820e-01, -2.06361294e-01,
         7.76072741e-02,  4.94316220e-02, -2.68267441e+00,
        -2.60976613e-01, -2.98366964e-01,  2.70217299e-01],
       [ 0.00000000e+00,  1.20100200e-01,  6.00826740e-02,
         0.00000000e+00,  2.23302841e-02,  2.15968227e+00,
        -6.49294853e-01, -3.75965297e-01,  3.68564069e-01],
       [ 0.00000000e+00,  0.00000000e+00,  2.27801323e-01,
         0.00000000e+00,  2.23302841e-02,  8.41687322e-02,
        -6.10104680e-01, -3.75965297e-01,  3.53635490e-01],
       [ 0.00000000e+00,  0.00000

{'cells': {'pdf': Array([[ 1.49220860e+00, -5.20337999e-01, -9.45273042e-01,
          -3.68194520e-01,  1.87361014e+00, -7.45318651e-01,
          -2.22784710e+00,  3.36281300e-01,  2.38622427e-02],
         [ 1.84728384e+00,  3.38659674e-01, -5.91143131e-01,
           3.75676441e+00,  9.76053476e-01, -1.69862556e+00,
           2.55079746e+00,  1.84629095e+00, -4.83699024e-01],
         [ 4.37712193e+00,  1.02265358e+00,  1.72587657e+00,
           1.43873715e+00,  7.31658518e-01,  2.82243967e+00,
           7.12082565e-01,  5.36072850e-01, -9.48111713e-02],
         [ 3.31386709e+00,  1.99662834e-01,  6.84733689e-01,
           2.19354010e+00,  8.95858526e-01, -1.94800234e+00,
           1.07538724e+00,  9.32697415e-01, -2.57070839e-01],
         [ 3.55649829e+00,  3.90677363e-01,  6.69080079e-01,
           2.05575323e+00,  8.68637085e-01,  8.58890265e-02,
           1.04170060e+00,  8.57935548e-01, -2.28038281e-01],
         [ 3.55649829e+00,  3.90677363e-01,  7.62826979e-01,
   

In [16]:
params = env.step(params,config)
params

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00, -7.55809975e+00,  2.17509365e+01,
         4.12495899e+00, -1.51574326e+00,  7.03882313e+00,
         1.41706142e+01,  1.23535144e+00,  6.88649368e+00],
       [ 0.00000000e+00, -8.58997703e-01,  9.32828140e+00,
        -2.31802726e+00, -3.82732213e-01,  3.88291073e+00,
        -3.94575667e+00, -3.25891948e+00,  1.90114212e+00],
       [ 0.00000000e+00, -6.83993936e-01, -1.57178366e+00,
         7.54802942e-01,  2.44556248e-01, -1.00111828e+01,
         2.51991928e-01, -1.64707810e-01,  3.15076470e-01],
       [ 0.00000000e+00,  8.22990775e-01,  1.01655698e+00,
        -1.37786865e-01, -3.51080298e-02,  7.91407204e+00,
        -1.22955561e+00, -1.05488622e+00,  1.11068833e+00],
       [ 0.00000000e+00, -1.91014528e-01,  1.01796389e+00,
         0.00000000e+00, -7.88658857e-03, -2.90426040e+00,
        -1.41786456e+00, -9.05362546e-01,  8.84217680e-01],
       [ 0.00000000e+00,  0.00000

{'cells': {'pdf': Array([[ 2.90028019e+01,  1.48571758e+01, -3.86172562e+01,
           2.28777599e+00, -1.09284697e+01, -9.36145592e+00,
          -1.99555702e+01, -2.75875425e+00, -1.17405949e+01],
         [-5.88757753e+00,  2.56509018e+00, -1.10867939e+01,
           1.15008593e+01, -3.62322927e-02, -2.38817811e+00,
           4.94124699e+00,  6.38307571e+00, -1.79700732e+00],
         [ 5.15890121e+00,  2.48795652e+00,  5.19351864e+00,
           5.89809418e-02,  2.19500661e-01,  1.05453014e+01,
           3.83486569e-01,  1.97320729e-01, -1.07893199e-01],
         [-1.78091860e+00, -3.01336169e-01, -1.88608456e+00,
           5.33350182e+00,  5.78938246e-01, -6.91795063e+00,
           1.80039001e+00,  2.99049973e+00, -1.13333702e+00],
         [ 3.03703547e+00,  4.53987360e-01, -7.34962821e-02,
           2.93575525e+00,  6.54018939e-01,  2.96835542e+00,
           2.08955503e+00,  1.48659730e+00, -7.29466915e-01],
         [ 2.59425855e+00,  2.21068293e-01,  2.18416095e-01,
   

In [17]:
for i in tqdm(range(10)):
    params = env.step(params,config)

  0%|          | 0/10 [00:00<?, ?it/s]

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00, -1.71200943e+01,  6.03743858e+01,
         9.21308327e+00,  1.03951082e+01,  3.89618206e+00,
         5.61177521e+01,  1.23291569e+01,  1.98488007e+01],
       [ 0.00000000e+00,  1.22920856e+01,  1.81554413e+01,
        -1.14418783e+01, -7.64685035e-01, -3.67816467e+01,
        -1.04903851e+01, -1.28604107e+01, -5.77956057e+00],
       [ 0.00000000e+00,  7.71336555e-02, -2.09114971e+01,
         5.27452087e+00,  1.62111425e+00, -1.78857079e+01,
         2.65519738e+00,  2.36465955e+00, -5.13851047e-01],
       [ 0.00000000e+00,  2.78929281e+00, -1.76055984e+01,
        -2.39774656e+00, -4.06916440e-01,  9.11887741e+01,
        -7.39122486e+00, -4.82670498e+00,  3.71054125e+00],
       [ 0.00000000e+00, -7.55323529e-01, -1.02361431e+01,
        -2.61053801e-01, -2.46614099e-01,  1.45796814e+01,
        -8.55968475e+00, -1.75753272e+00,  1.79410958e+00],
       [ 0.00000000e+00,  2.32919

 30%|███       | 3/10 [00:00<00:00, 24.29it/s]

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00, -3.48001556e+01,  1.17530518e+02,
         8.62473907e+01,  4.98282242e+00,  5.17096748e+01,
         1.66596603e+02,  7.74435501e+01,  4.37869263e+01],
       [ 0.00000000e+00, -9.36742020e+00,  2.04879837e+01,
        -7.68021240e+01, -2.88198471e-01, -1.64435333e+02,
        -3.11501923e+01, -1.17007805e+02, -1.07879620e+01],
       [ 0.00000000e+00,  2.01029167e+01, -5.39639435e+01,
        -7.87758713e+01, -4.49687004e+00,  5.82454681e-01,
         2.38589554e+01, -7.42420654e+01,  1.02390379e-01],
       [ 0.00000000e+00,  1.12779831e+02, -1.80903473e+01,
         8.85503387e+01, -3.31176071e+01,  4.34096313e+02,
        -3.43469315e+01,  1.39649567e+02, -2.11594200e+00],
       [ 0.00000000e+00, -1.05658363e+02,  2.45808411e+01,
        -2.48252130e+00, -1.24200130e+00, -1.79767136e+02,
        -4.45142441e+01, -7.78597355e-01,  9.28964520e+00],
       [ 0.00000000e+00,  1.06624

 60%|██████    | 6/10 [00:00<00:00, 23.74it/s]

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00, -1.15521973e+03,  3.20150830e+03,
         3.53255737e+03,  1.96915894e+03,  2.16243823e+03,
         2.02441250e+04,  5.30152344e+03,  2.31416162e+03],
       [ 0.00000000e+00,  9.09838867e+00, -1.00109688e+04,
         3.31078784e+03, -2.81627930e+02, -1.29311973e+04,
        -7.33618262e+03,  7.29459619e+03, -2.18957690e+03],
       [ 0.00000000e+00,  6.00565613e+02,  9.74113525e+02,
        -2.18127637e+03, -2.36920215e+03, -4.50189648e+03,
         1.08795166e+03, -1.84177910e+04,  2.27364990e+03],
       [ 0.00000000e+00,  4.36129150e+03,  1.00288174e+04,
        -2.08844849e+02, -1.67433423e+03,  3.53760859e+04,
         9.47912988e+03,  1.40836152e+04, -9.06806335e+02],
       [ 0.00000000e+00, -1.20675488e+04, -2.36477695e+04,
        -2.02674731e+03,  1.09656738e+02, -7.88008203e+04,
        -2.42920371e+04, -6.06456152e+03,  7.55638428e+02],
       [ 0.00000000e+00,  9.90849

 90%|█████████ | 9/10 [00:00<00:00, 24.05it/s]

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00, -1.66135596e+03,  6.48492090e+03,
         8.12923730e+03,  6.57438330e+03,  5.94349902e+03,
         6.48129492e+04,  1.64922461e+04,  5.46237695e+03],
       [ 0.00000000e+00, -1.34157031e+03, -5.06985234e+04,
         1.17772764e+04, -1.86312195e+03, -5.75611797e+04,
        -2.20851680e+04,  2.70349062e+04, -3.02956250e+03],
       [ 0.00000000e+00,  3.21246729e+03,  2.88009668e+03,
        -5.66476709e+03, -5.53306006e+03, -1.51468945e+04,
        -5.99746924e+03, -6.07040547e+04,  4.35768018e+03],
       [ 0.00000000e+00,  1.06254170e+04,  3.60472969e+04,
        -3.56349927e+03, -3.29275952e+03,  1.22938016e+05,
         5.73588984e+04,  5.38473672e+04, -6.17637305e+03],
       [ 0.00000000e+00, -3.72250312e+04, -1.10485930e+05,
         1.91310229e+03, -1.40088245e+03, -2.99684844e+05,
        -1.20879531e+05, -2.76708867e+04,  7.47076465e+03],
       [ 0.00000000e+00,  3.55454

100%|██████████| 10/10 [00:00<00:00, 24.21it/s]

sum: Traced<ShapedArray(float32[9])>with<BatchTrace(level=1/0)> with
  val = Array([[ 0.00000000e+00,  1.40997094e+05,  1.05184047e+05,
        -5.34040312e+04,  2.24664562e+05,  6.18402000e+05,
         1.66430050e+06,  1.78724362e+06, -1.84800016e+05],
       [ 0.00000000e+00, -2.12205016e+05, -3.61123950e+06,
         7.01237000e+05, -2.34016297e+05, -4.46059200e+06,
         4.30707812e+05, -7.87587875e+05,  6.00299875e+05],
       [ 0.00000000e+00,  2.04271609e+05,  7.81548625e+05,
        -2.73324156e+05, -2.60650406e+05,  1.12728650e+06,
        -3.58093300e+06, -1.47089550e+06, -6.97503188e+05],
       [ 0.00000000e+00,  6.39960156e+04,  2.13559650e+06,
        -7.51266750e+05,  1.51274938e+05,  4.12367225e+06,
         8.17715050e+06,  2.62528425e+06,  2.11468938e+05],
       [ 0.00000000e+00, -6.59339125e+05, -9.32542000e+06,
         1.79967538e+06, -7.08002375e+05, -1.51316490e+07,
        -1.07825470e+07, -2.34568700e+06,  6.39910188e+05],
       [ 0.00000000e+00,  1.17922

In [18]:
params

{'cells': {'pdf': Array([[-9.27198203e+04, -1.78767750e+05, -4.48306750e+05,
          -9.05053250e+05, -1.21139555e+05, -5.69579312e+05,
          -1.96016750e+06, -1.80769388e+06,  8.43275781e+04],
         [-2.03790850e+06,  8.59464000e+05,  6.04368800e+06,
          -1.23136338e+06,  5.39561812e+05,  5.67913550e+06,
          -3.18885750e+05,  1.37059725e+06, -7.28321938e+05],
         [-6.69080125e+05,  6.80265625e+03, -1.01383419e+06,
           1.66147312e+06,  3.57191719e+05, -9.20478750e+05,
           3.61532250e+06,  1.93014438e+06,  6.37479250e+05],
         [-8.08196625e+05,  2.20573750e+04, -5.16782950e+06,
           5.57082500e+04, -3.63172531e+05, -4.34412150e+06,
          -8.86727600e+06, -2.43889525e+06, -3.87075781e+05],
         [-2.07242575e+06,  1.25124862e+06,  1.87217240e+07,
          -2.87962175e+06,  1.82874462e+06,  1.75747680e+07,
           1.18616250e+07,  2.99628300e+06, -6.44443938e+05],
         [ 5.32859700e+06, -2.64204450e+06, -2.19296580e+07,
   